# Setup IAM for Kinesis

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

# Create Kinesis Role

In [2]:
iam_kinesis_role_name = 'DSOAWS_Kinesis'

In [3]:
iam_kinesis_role_passed = False

In [4]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesis.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "firehose.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "kinesisanalytics.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }            
  ]
} 

In [5]:
import json
import time

from botocore.exceptions import ClientError

try:
    iam_role_kinesis = iam.create_role(
        RoleName=iam_kinesis_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Kinesis Role'
    )
    print('Role succesfully created.')
    iam_kinesis_role_passed = True
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_kinesis = iam.get_role(RoleName=iam_kinesis_role_name)
        print('Role already exists. That is OK.')
        iam_kinesis_role_passed = True
    else:
        print('Unexpected error: %s' % e)
        
time.sleep(30)

Role succesfully created.


In [6]:
iam_role_kinesis_name = iam_role_kinesis['Role']['RoleName']
print('Role Name: {}'.format(iam_role_kinesis_name))

Role Name: DSOAWS_Kinesis


In [7]:
iam_role_kinesis_arn = iam_role_kinesis['Role']['Arn']
print('Role ARN: {}'.format(iam_role_kinesis_arn))

Role ARN: arn:aws:iam::085964654406:role/DSOAWS_Kinesis


In [8]:
account_id = sts.get_caller_identity()['Account']

# Specify Stream Name

In [9]:
stream_name = 'dsoaws-kinesis-data-stream'

# Specify Firehose Name

In [10]:
firehose_name = 'dsoaws-kinesis-data-firehose'

# Specify Lambda Function Name

In [11]:
lambda_fn_name = 'DeliverKinesisAnalyticsToCloudWatch'

# Create Policy

In [12]:
kinesis_policy_doc = {
    
    "Version": "2012-10-17",
    "Statement": [
        {      
            "Effect": "Allow",      
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject"
            ],      
            "Resource": [        
                "arn:aws:s3:::{}/kinesis-data-firehose".format(bucket),
                "arn:aws:s3:::{}/kinesis-data-firehose/*".format(bucket)
            ]    
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:PutLogEvents"
            ],
            "Resource": [
                "arn:aws:logs:{}:{}:log-group:/*".format(region, account_id)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesis:Get*",
                "kinesis:DescribeStream",
                "kinesis:Put*",
                "kinesis:List*",
            ],
            "Resource": [
                "arn:aws:kinesis:{}:{}:stream/{}".format(region, account_id, stream_name)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "firehose:*",
            ],
            "Resource": [
                "arn:aws:firehose:{}:{}:deliverystream/{}".format(region, account_id, firehose_name)
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "kinesisanalytics:*",
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Resource": "arn:aws:lambda:{}:{}:function:{}:$LATEST".format(region, account_id, lambda_fn_name)
        },
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::*:role/service-role/kinesis-analytics*"
        }
    ]
}

print(json.dumps(kinesis_policy_doc, indent=4, sort_keys=True, default=str))

{
    "Statement": [
        {
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:PutObject"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:s3:::sagemaker-us-west-2-085964654406/kinesis-data-firehose",
                "arn:aws:s3:::sagemaker-us-west-2-085964654406/kinesis-data-firehose/*"
            ]
        },
        {
            "Action": [
                "logs:PutLogEvents"
            ],
            "Effect": "Allow",
            "Resource": [
                "arn:aws:logs:us-west-2:085964654406:log-group:/*"
            ]
        },
        {
            "Action": [
                "kinesis:Get*",
                "kinesis:DescribeStream",
                "kinesis:Put*",
                "kinesis:List*"
            ],
            "Effect": "A

# Update Policy

In [13]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_kinesis_name,
    PolicyName='DSOAWS_KinesisPolicy',
    PolicyDocument=json.dumps(kinesis_policy_doc)
)

time.sleep(30)

In [14]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "206",
            "content-type": "text/xml",
            "date": "Sat, 26 Sep 2020 20:28:44 GMT",
            "x-amzn-requestid": "99f32939-75d7-4cca-9e6b-675e7503b973"
        },
        "HTTPStatusCode": 200,
        "RequestId": "99f32939-75d7-4cca-9e6b-675e7503b973",
        "RetryAttempts": 0
    }
}


# Create AWS Lambda IAM Role

In [15]:
iam_lambda_role_name = 'DSOAWS_Lambda'

In [16]:
iam_lambda_role_passed = False

In [17]:
assume_role_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "kinesisanalytics.amazonaws.com"
        },
            "Action": "sts:AssumeRole"
        }
    ]
}

In [18]:
import time

from botocore.exceptions import ClientError

try:
    iam_role_lambda = iam.create_role(
        RoleName=iam_lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Lambda Role'
    )
    print('Role succesfully created.')
    iam_lambda_role_passed = True
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_lambda = iam.get_role(RoleName=iam_lambda_role_name)
        print('Role already exists. This is OK.')
        iam_lambda_role_passed = True
    else:
        print('Unexpected error: %s' % e)
        
time.sleep(30)

Role succesfully created.


In [19]:
iam_role_lambda_name = iam_role_lambda['Role']['RoleName']
print('Role Name: {}'.format(iam_role_lambda_name))

Role Name: DSOAWS_Lambda


In [20]:
iam_role_lambda_arn = iam_role_lambda['Role']['Arn']
print('Role ARN: {}'.format(iam_role_lambda_arn))

Role ARN: arn:aws:iam::085964654406:role/DSOAWS_Lambda


# Create AWS Lambda IAM Policy

In [21]:
lambda_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "UseLambdaFunction",
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Resource": "arn:aws:lambda:{}:{}:function:*".format(region, account_id)
        },
        {
            "Effect": "Allow",
            "Action": "cloudwatch:*",
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:{}:{}:*".format(region, account_id)
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:{}:{}:log-group:/aws/lambda/*".format(region, account_id)
        }
    ]
}

In [22]:
print(json.dumps(lambda_policy_doc, indent=4, sort_keys=True, default=str))

{
    "Statement": [
        {
            "Action": [
                "lambda:InvokeFunction",
                "lambda:GetFunctionConfiguration"
            ],
            "Effect": "Allow",
            "Resource": "arn:aws:lambda:us-west-2:085964654406:function:*",
            "Sid": "UseLambdaFunction"
        },
        {
            "Action": "cloudwatch:*",
            "Effect": "Allow",
            "Resource": "*"
        },
        {
            "Action": "logs:CreateLogGroup",
            "Effect": "Allow",
            "Resource": "arn:aws:logs:us-west-2:085964654406:*"
        },
        {
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Effect": "Allow",
            "Resource": "arn:aws:logs:us-west-2:085964654406:log-group:/aws/lambda/*"
        }
    ],
    "Version": "2012-10-17"
}


In [23]:
import time

response = iam.put_role_policy(
    RoleName=iam_role_lambda_name,
    PolicyName='DSOAWS_LambdaPolicy',
    PolicyDocument=json.dumps(lambda_policy_doc)
)

time.sleep(30)

In [24]:
print(json.dumps(response, indent=4, sort_keys=True, default=str))

{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "206",
            "content-type": "text/xml",
            "date": "Sat, 26 Sep 2020 20:29:44 GMT",
            "x-amzn-requestid": "e05c6342-2335-4033-a358-1e8c761a19ec"
        },
        "HTTPStatusCode": 200,
        "RequestId": "e05c6342-2335-4033-a358-1e8c761a19ec",
        "RetryAttempts": 0
    }
}


# Store Variables for Next Notebooks

In [25]:
%store stream_name

Stored 'stream_name' (str)


In [26]:
%store firehose_name

Stored 'firehose_name' (str)


In [27]:
%store iam_kinesis_role_name

Stored 'iam_kinesis_role_name' (str)


In [28]:
%store iam_role_kinesis_arn

Stored 'iam_role_kinesis_arn' (str)


In [29]:
%store iam_lambda_role_name

Stored 'iam_lambda_role_name' (str)


In [30]:
%store iam_role_lambda_arn

Stored 'iam_role_lambda_arn' (str)


In [31]:
%store lambda_fn_name

Stored 'lambda_fn_name' (str)


In [32]:
%store iam_kinesis_role_passed

Stored 'iam_kinesis_role_passed' (bool)


In [33]:
%store iam_lambda_role_passed

Stored 'iam_lambda_role_passed' (bool)


In [34]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                               -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-west-2-085964654406/data/amazon
balance_dataset                                       -> True
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-160114585
firehose_name                                         -> 'dsoaws-kinesis-data-firehose'
iam_kinesis_role_name                                 -> 'DSOAWS_Kinesis'
iam_kinesis_role_passed                               -> True
iam_lambda_role_name                                  -> 'DSOAWS_Lambda'
iam_lambda_role_passed                                -> True
iam_role_kinesis_arn                                  -> 'arn:aws:iam::085964654406:role/DSOAWS_Kinesis'
iam_role_lambda_arn                                 

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();